# Vectorize a raster

In [1]:
import rasterio
from rasterio.features import shapes
from shapely.geometry import shape

from geopandas import GeoDataFrame
from pandas import DataFrame


with rasterio.open('optima.tif') as src:
    data = src.read(1, masked=False)

    # Use a generator instead of a list
    shape_gen = ((shape(s), v) for s, v in shapes(data, transform=src.transform))

    # build a dict from unpacked shapes
    gdf = GeoDataFrame(dict(zip(["geometry", "class"], zip(*shape_gen))), crs=src.crs)


# Dump to file

In [ ]:
gdf.to_file('vectorized.geojson', driver="GeoJSON")

# Resampling

In [ ]:
import rasterio
from rasterio.enums import Resampling

upscale_factor = 0.3

with rasterio.open("optima.tif") as dataset:

    # resample data to target shape
    data = dataset.read(
        out_shape=(
            dataset.count,
            int(dataset.height * upscale_factor),
            int(dataset.width * upscale_factor)
        ),
        resampling=Resampling.bilinear
    )

    # scale image transform
    transform = dataset.transform * dataset.transform.scale(
        (dataset.width / data.shape[-1]),
        (dataset.height / data.shape[-2])
    )